In [1]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf

/home/yicheng/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_pickle('df_daily_interpolated.pickle')
df = df.dropna()

In [3]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len):
    labels = df['BTC_weighted_mean'].as_matrix()
    
    data_raw = df.as_matrix() # convert to numpy array
    data_raw = preprocessing.minmax_scale(data_raw) # normalizing features
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len])
    
    data = np.array(data);
    
    valid_set_size_percentage = 20 
    test_set_size_percentage = 10 
    
    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:] # cannot see last day, which we aim to predict
    y_train = labels[seq_len-1:train_set_size+seq_len-1]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:]
    y_valid = labels[seq_len-1+train_set_size:train_set_size+valid_set_size+seq_len-1]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:]
    y_test = labels[seq_len-1+train_set_size+valid_set_size:]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [4]:
# create train, test data
seq_len = 10 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df, seq_len)
y_train = y_train.reshape([-1,1])
y_valid = y_valid.reshape([-1,1])
y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (410, 9, 18)
y_train.shape =  (410, 1)
x_valid.shape =  (117, 9, 18)
y_valid.shape =  (117, 1)
x_test.shape =  (58, 9, 18)
y_test.shape =  (58, 1)


In [5]:
# shuffle training data
s = np.arange(x_train.shape[0])
np.random.shuffle(s)
x_train = x_train[s]
y_train = y_train[s]

In [6]:
def SMAPE(y, y_pred):
    if len(y) != len(y_pred):
        raise ValueError('Length of prediction array is not equal to length of y array.')
    return np.mean(np.abs(y-y_pred)*2/(np.abs(y)+np.abs(y_pred)))

## LSTM model graph

In [7]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = x_train.shape[-1]
n_neurons = 300  # cell.state_size
n_outputs = 1
n_layers = 1
learning_rate = 0.0001
batch_size = 50
n_epochs = 200 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
keep_prob = 0.85
max_gradient_norm = 2

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# use Basic RNN Cell
layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, \
                                  initializer=tf.contrib.layers.xavier_initializer(), \
                                  activation=tf.nn.elu)
         for layer in range(n_layers)]

# use Basic LSTM Cell 
# layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#           for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.nn.dropout(outputs, keep_prob)

# 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
stacked_outputs = tf.reshape(outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence

loss = tf.reduce_mean(tf.square(tf.div(outputs,y) - 1)) # loss function = mean squared error 
# loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
#loss = tf.reduce_mean(tf.abs(y-outputs)*2/(tf.abs(y)+tf.abs(outputs))) # SMAPE
params = tf.trainable_variables()
gradients = tf.gradients(loss, params)
gradient_norm = tf.global_norm(gradients)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
param_norm = tf.global_norm(params)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
# training_op = optimizer.minimize(loss)
training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

# initialize parameters
sess = tf.Session()
global_step = tf.Variable(0, name='global_step', trainable=False)
saver = tf.train.Saver(max_to_keep=2)

Instructions for updating:
Use the retry module or similar alternatives.


In [9]:
# run graph
ckpt = tf.train.get_checkpoint_state('LSTM')
v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
#     iteration = tf.get_variable('iteration',[1])
    saver.restore(sess, ckpt.model_checkpoint_path)
    iteration = global_step.eval(session=sess) # get last global_step
    print("Start from iteration:", iteration)
else:
    print('There is not saved parameters. Creating model with fresh parameters.')
#     iteration = tf.get_variable('iteration',[1], initializer = tf.zeros_initializer)
    sess.run(tf.global_variables_initializer())
    iteration = 0
#     sess.run(iteration)
    
while True:
    iteration = iteration + 1
    x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
     
    # output_feed contains the things we want to fetch.
    output_feed = [training_op, loss, param_norm, gradient_norm]
    [_, loss_val, param_norm_val, gradient_norm_val] \
        = sess.run(output_feed, feed_dict={X: x_batch, y: y_batch})
    
    if iteration % int(10*train_set_size/batch_size) == 0:
#             loss_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
#             loss_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
        y_train_pred = sess.run(outputs, feed_dict={X: x_train})
        y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
        print('%.2f epochs: loss = %.5f, param_norm = %.3f, gradient_norm = %.3f, SMAPE train/valid = %.6f/%.6f' \
              %(iteration*batch_size/train_set_size, loss_val, param_norm_val, 
                gradient_norm_val, \
                SMAPE(y_train, y_train_pred), \
                SMAPE(y_valid, y_valid_pred)))
#             print('%.2f epochs: loss train/valid = %.6f/%.6f'%(
#                 iteration*batch_size/train_set_size, loss_train, loss_valid))
    if iteration % int(50*train_set_size/batch_size) == 0:
        global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
        save_path = saver.save(sess, "./LSTM/model.ckpt", global_step=global_step)
        print('Saved parameters to %s' % save_path)

y_train_pred = sess.run(outputs, feed_dict={X: x_train})
y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
y_test_pred = sess.run(outputs, feed_dict={X: x_test})

There is not saved parameters. Creating model with fresh parameters.
10.00 epochs: loss = 0.94127, param_norm = 22.979, gradient_norm = 0.236, SMAPE train/valid = 1.859854/1.873632
20.00 epochs: loss = 0.03224, param_norm = 23.765, gradient_norm = 0.045, SMAPE train/valid = 0.165959/0.414150
30.00 epochs: loss = 0.02644, param_norm = 23.781, gradient_norm = 0.041, SMAPE train/valid = 0.163941/0.406823
40.00 epochs: loss = 0.02680, param_norm = 23.807, gradient_norm = 0.044, SMAPE train/valid = 0.156581/0.384143
50.00 epochs: loss = 0.02780, param_norm = 23.835, gradient_norm = 0.047, SMAPE train/valid = 0.156518/0.377041
Saved parameters to ./LSTM/model.ckpt-410
60.00 epochs: loss = 0.02376, param_norm = 23.866, gradient_norm = 0.051, SMAPE train/valid = 0.146964/0.360569
70.00 epochs: loss = 0.03137, param_norm = 23.896, gradient_norm = 0.209, SMAPE train/valid = 0.145016/0.353118
80.00 epochs: loss = 0.03127, param_norm = 23.937, gradient_norm = 0.301, SMAPE train/valid = 0.138403/0.

690.00 epochs: loss = 0.00726, param_norm = 25.804, gradient_norm = 0.443, SMAPE train/valid = 0.067568/0.330652
700.00 epochs: loss = 0.00609, param_norm = 25.818, gradient_norm = 0.260, SMAPE train/valid = 0.063897/0.329426
Saved parameters to ./LSTM/model.ckpt-5740
710.00 epochs: loss = 0.00755, param_norm = 25.822, gradient_norm = 0.157, SMAPE train/valid = 0.059889/0.336139
720.00 epochs: loss = 0.00437, param_norm = 25.829, gradient_norm = 0.071, SMAPE train/valid = 0.061882/0.314677
730.00 epochs: loss = 0.00399, param_norm = 25.840, gradient_norm = 0.278, SMAPE train/valid = 0.060757/0.341609
740.00 epochs: loss = 0.00528, param_norm = 25.853, gradient_norm = 0.210, SMAPE train/valid = 0.062698/0.342571
750.00 epochs: loss = 0.00721, param_norm = 25.857, gradient_norm = 0.257, SMAPE train/valid = 0.061026/0.347704
Saved parameters to ./LSTM/model.ckpt-6150
760.00 epochs: loss = 0.00862, param_norm = 25.858, gradient_norm = 0.510, SMAPE train/valid = 0.064454/0.355931
770.00 epo

1360.00 epochs: loss = 0.00676, param_norm = 26.226, gradient_norm = 0.559, SMAPE train/valid = 0.065541/0.521889
1370.00 epochs: loss = 0.00363, param_norm = 26.227, gradient_norm = 0.207, SMAPE train/valid = 0.052131/0.498913
1380.00 epochs: loss = 0.00419, param_norm = 26.222, gradient_norm = 0.495, SMAPE train/valid = 0.051389/0.493359
1390.00 epochs: loss = 0.00699, param_norm = 26.226, gradient_norm = 0.675, SMAPE train/valid = 0.056300/0.485644
1400.00 epochs: loss = 0.00703, param_norm = 26.227, gradient_norm = 0.700, SMAPE train/valid = 0.053586/0.496820
Saved parameters to ./LSTM/model.ckpt-11480
1410.00 epochs: loss = 0.00370, param_norm = 26.228, gradient_norm = 0.309, SMAPE train/valid = 0.051693/0.495046
1420.00 epochs: loss = 0.00518, param_norm = 26.226, gradient_norm = 0.326, SMAPE train/valid = 0.059981/0.505871
1430.00 epochs: loss = 0.00425, param_norm = 26.235, gradient_norm = 0.335, SMAPE train/valid = 0.051507/0.501098
1440.00 epochs: loss = 0.00489, param_norm =

2030.00 epochs: loss = 0.00459, param_norm = 26.432, gradient_norm = 0.950, SMAPE train/valid = 0.058238/0.586368
2040.00 epochs: loss = 0.00450, param_norm = 26.440, gradient_norm = 0.341, SMAPE train/valid = 0.050577/0.573746
2050.00 epochs: loss = 0.00546, param_norm = 26.440, gradient_norm = 0.853, SMAPE train/valid = 0.047350/0.562110
Saved parameters to ./LSTM/model.ckpt-16810
2060.00 epochs: loss = 0.00517, param_norm = 26.445, gradient_norm = 0.100, SMAPE train/valid = 0.045208/0.556095
2070.00 epochs: loss = 0.00391, param_norm = 26.446, gradient_norm = 0.343, SMAPE train/valid = 0.045560/0.554562
2080.00 epochs: loss = 0.00589, param_norm = 26.456, gradient_norm = 0.190, SMAPE train/valid = 0.049150/0.570209
2090.00 epochs: loss = 0.00495, param_norm = 26.458, gradient_norm = 0.311, SMAPE train/valid = 0.051764/0.567711
2100.00 epochs: loss = 0.00278, param_norm = 26.460, gradient_norm = 0.397, SMAPE train/valid = 0.046639/0.567461
Saved parameters to ./LSTM/model.ckpt-17220


2700.00 epochs: loss = 0.00410, param_norm = 26.737, gradient_norm = 0.200, SMAPE train/valid = 0.042386/0.581805
Saved parameters to ./LSTM/model.ckpt-22140
2710.00 epochs: loss = 0.00245, param_norm = 26.747, gradient_norm = 0.150, SMAPE train/valid = 0.044530/0.573394
2720.00 epochs: loss = 0.00472, param_norm = 26.748, gradient_norm = 0.238, SMAPE train/valid = 0.045232/0.562533
2730.00 epochs: loss = 0.00283, param_norm = 26.749, gradient_norm = 0.159, SMAPE train/valid = 0.044783/0.554516
2740.00 epochs: loss = 0.00498, param_norm = 26.757, gradient_norm = 0.411, SMAPE train/valid = 0.045478/0.557189
2750.00 epochs: loss = 0.00462, param_norm = 26.765, gradient_norm = 1.023, SMAPE train/valid = 0.054180/0.551319
Saved parameters to ./LSTM/model.ckpt-22550
2760.00 epochs: loss = 0.00367, param_norm = 26.761, gradient_norm = 0.385, SMAPE train/valid = 0.049702/0.558826
2770.00 epochs: loss = 0.00290, param_norm = 26.762, gradient_norm = 0.158, SMAPE train/valid = 0.049155/0.568539


...

14750.00 epochs: loss = 0.00093, param_norm = 39.736, gradient_norm = 0.178, SMAPE train/valid = 0.028642/0.389582
Saved parameters to ./LSTM/model.ckpt-120950
14760.00 epochs: loss = 0.00153, param_norm = 39.738, gradient_norm = 0.188, SMAPE train/valid = 0.034735/0.399237
14770.00 epochs: loss = 0.00116, param_norm = 39.740, gradient_norm = 0.082, SMAPE train/valid = 0.028178/0.388129
14780.00 epochs: loss = 0.00285, param_norm = 39.757, gradient_norm = 0.309, SMAPE train/valid = 0.030704/0.389418
14790.00 epochs: loss = 0.00123, param_norm = 39.765, gradient_norm = 0.042, SMAPE train/valid = 0.030584/0.390826
14800.00 epochs: loss = 0.00112, param_norm = 39.782, gradient_norm = 0.039, SMAPE train/valid = 0.027643/0.385196
Saved parameters to ./LSTM/model.ckpt-121360
14810.00 epochs: loss = 0.00222, param_norm = 39.787, gradient_norm = 0.239, SMAPE train/valid = 0.033284/0.408666
14820.00 epochs: loss = 0.00138, param_norm = 39.810, gradient_norm = 0.085, SMAPE train/valid = 0.027137

15410.00 epochs: loss = 0.00211, param_norm = 40.336, gradient_norm = 0.284, SMAPE train/valid = 0.029365/0.382020
15420.00 epochs: loss = 0.00178, param_norm = 40.348, gradient_norm = 0.065, SMAPE train/valid = 0.030111/0.412129
15430.00 epochs: loss = 0.00144, param_norm = 40.352, gradient_norm = 0.106, SMAPE train/valid = 0.030039/0.401818
15440.00 epochs: loss = 0.00143, param_norm = 40.352, gradient_norm = 0.103, SMAPE train/valid = 0.028553/0.391844
15450.00 epochs: loss = 0.00160, param_norm = 40.361, gradient_norm = 0.209, SMAPE train/valid = 0.029319/0.380020
Saved parameters to ./LSTM/model.ckpt-126690
15460.00 epochs: loss = 0.00104, param_norm = 40.364, gradient_norm = 0.074, SMAPE train/valid = 0.028607/0.402385
15470.00 epochs: loss = 0.00117, param_norm = 40.383, gradient_norm = 0.120, SMAPE train/valid = 0.030809/0.401809
15480.00 epochs: loss = 0.00153, param_norm = 40.387, gradient_norm = 0.329, SMAPE train/valid = 0.029981/0.382787
15490.00 epochs: loss = 0.00135, pa

16080.00 epochs: loss = 0.00132, param_norm = 40.907, gradient_norm = 0.097, SMAPE train/valid = 0.027180/0.394864
16090.00 epochs: loss = 0.00149, param_norm = 40.914, gradient_norm = 0.186, SMAPE train/valid = 0.029946/0.389127
16100.00 epochs: loss = 0.00121, param_norm = 40.925, gradient_norm = 0.248, SMAPE train/valid = 0.027625/0.403669
Saved parameters to ./LSTM/model.ckpt-132020
16110.00 epochs: loss = 0.00145, param_norm = 40.933, gradient_norm = 0.065, SMAPE train/valid = 0.027933/0.403319
16120.00 epochs: loss = 0.00161, param_norm = 40.946, gradient_norm = 0.125, SMAPE train/valid = 0.028457/0.414797
16130.00 epochs: loss = 0.00151, param_norm = 40.971, gradient_norm = 0.231, SMAPE train/valid = 0.029972/0.410809
16140.00 epochs: loss = 0.00106, param_norm = 40.982, gradient_norm = 0.079, SMAPE train/valid = 0.029206/0.410391
16150.00 epochs: loss = 0.00150, param_norm = 40.995, gradient_norm = 0.239, SMAPE train/valid = 0.027267/0.413534
Saved parameters to ./LSTM/model.ck

16750.00 epochs: loss = 0.00072, param_norm = 41.527, gradient_norm = 0.078, SMAPE train/valid = 0.030419/0.415560
Saved parameters to ./LSTM/model.ckpt-137350
16760.00 epochs: loss = 0.00182, param_norm = 41.533, gradient_norm = 0.170, SMAPE train/valid = 0.027877/0.419923
16770.00 epochs: loss = 0.00109, param_norm = 41.545, gradient_norm = 0.065, SMAPE train/valid = 0.031140/0.422250
16780.00 epochs: loss = 0.00110, param_norm = 41.550, gradient_norm = 0.068, SMAPE train/valid = 0.029337/0.407292
16790.00 epochs: loss = 0.00151, param_norm = 41.559, gradient_norm = 0.087, SMAPE train/valid = 0.027202/0.393113
16800.00 epochs: loss = 0.00166, param_norm = 41.567, gradient_norm = 0.228, SMAPE train/valid = 0.026543/0.416895
Saved parameters to ./LSTM/model.ckpt-137760
16810.00 epochs: loss = 0.00114, param_norm = 41.578, gradient_norm = 0.061, SMAPE train/valid = 0.027830/0.413309
16820.00 epochs: loss = 0.00092, param_norm = 41.583, gradient_norm = 0.081, SMAPE train/valid = 0.027542

KeyboardInterrupt: 